In [1]:
import pickle
import numpy as np
import tensorflow as tf
import keras

import os
import datetime

from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D
from Chess.view import view_board_colour
from Chess.state import Board, construct_board
from Chess.constants import WHITE, BLACK
from sklearn.model_selection import train_test_split

from ipywidgets import interact, fixed
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
np.random.seed(20)

In [3]:
%load_ext tensorboard

In [4]:
# From https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
def new_tensorboard_run():
    logdir = os.path.join("tensorboard_logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Basic sequential model

We will begin training a basic fully-connected network specified with Keras. This is primarily meant as a test and to define some useful functions (such as those for visualising our outputs). We will start by preprocessing, then defining a model, training and evaluating.

### Data generation
The script `generate_data.py` will open a list of PGN (portable game notation) chess games and run through them, getting from it some state during the game. This state contains information on the location, type and colour of the pieces as well as the legal moves in the position. However, it does not have any information on the strength of those moves, meaning we can only predict the *legal* moves and not rank them.

The games which are generated are saved in a `txt` file as FEN (Forsyth-Edwards Notation) strings, which encapsulate all the information needed to recreate a position in chess. We can use these strings to reconstruct the boards and gain access to the information we need. 

The current file being used was obtained from Lichess and is a set of ~121,000 games played online over the course of 2013. They are a mixture of classical, blitz and bullet games played at a variety of different levels.

In [5]:
with open("generated_data/lichess_db_standard_rated_2013-01.txt", 'r') as f:
    all_boards = f.readlines()
    
# Take a random sample from boards to reduce the training time
idx = np.random.randint(0, len(all_boards), 5000)
all_boards = np.array(all_boards)
boards = all_boards[idx]

board_list = [construct_board(i) for i in boards]

In [6]:
# An example of what one of these positions looks like
view_board_colour(board_list[0])

Black to move - turn 9
 R     B  Q  K  B     Q 
 P  P  P        P     P 
                N  P    
          P        B    
       P     P          
                        
 P  P           P  P  P 
 R  N        K  B     R 



## Data processing
Now the data needs to be formed into a reasonable type for training. This section will get all the positions which can be moved to as a list for each game, and all the positions of the pieces for each board as training data. Initally, this won't account for the piece type, which side is to move, or the different types of pieces and moves.

In [39]:
# Uses set notation to get only unique moves
valid_move_list = [list({i for i in board.moves.all_valid}) for board in board_list] 

# Get all the piece types and positions from the location map of each board (training data)
piece_position_list = [{position: piece.kind for position, piece in board.loc_map.items()} for board in board_list]

In [40]:
# Encode the location of any piece as a 1, and all other positions as 0
feature = []
for instance in piece_position_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    feature.append(array.flatten())

In [41]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
target = []
for instance in valid_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    target.append(array.flatten())

In [42]:
X = np.array(feature)
Y = np.array(target)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [43]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(4000, 64)
(4000, 64)
(1000, 64)
(1000, 64)


Beginning with a simple sequential model, which takes in a 1x64 array of features and outputs a prediction at each of the 64 squares.

In [60]:
class TFModel():
    def __init__(self, name, n_epochs=200, batch_size=50, loss='binary_crossentropy', optimizer='adam'):
        self.name = name
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.loss_func = loss
        self.optimizer = optimizer
        logdir = os.path.join("tensorboard_logs", self.name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        self.tb_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    def compile(self):
        self.model.compile(
            loss=self.loss_func,
            optimizer=self.optimizer,
            metrics=["accuracy", "binary_accuracy"]
        )
    
    @property
    def summary(self):
        return self.model.summary
        
    def fit(self, X, Y):
        return self.model.fit(
            X, 
            Y, 
            epochs=self.n_epochs, 
            batch_size=self.batch_size, 
            callbacks=[self.tb_callback]
        )
    
    def evaluate(self, X, Y):
        self.model.evaluate(
            X, 
            Y, 
            callbacks=[self.tb_callback]
        )
        

In [61]:
fully_connected_model = TFModel("fully_connected")
fully_connected_model.model = model
fully_connected_model.compile()
fully_connected_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_10 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


2021-12-13 20:20:03.135560: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:20:03.135568: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:20:03.135820: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [62]:
fully_connected_model.fit(X_train, Y_train)

Epoch 1/200
11/80 [===>..........................] - ETA: 0s - loss: 0.2111 - accuracy: 0.0680 - binary_accuracy: 0.9154

2021-12-13 20:20:04.317377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 20:20:04.417998: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:20:04.418008: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:20:04.444595: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 20:20:04.445036: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 20:20:04.445820: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/fully_connected_20211213-202003/train/plugins/profile/2021_12_13_20_20_04
2021-12-13 20:20:04.446303: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/fully_connected_20211213-202003/train/plugi

80/80 [==============================] - 1s 7ms/step - loss: 0.2095 - accuracy: 0.0766 - binary_accuracy: 0.9167
Epoch 2/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2038 - accuracy: 0.0887 - binary_accuracy: 0.9206
Epoch 3/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2039 - accuracy: 0.0787 - binary_accuracy: 0.9200
Epoch 4/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2018 - accuracy: 0.0843 - binary_accuracy: 0.9222
Epoch 5/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1997 - accuracy: 0.0894 - binary_accuracy: 0.9226
Epoch 6/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2010 - accuracy: 0.0873 - binary_accuracy: 0.9209
Epoch 7/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2020 - accuracy: 0.0859 - binary_accuracy: 0.9207
Epoch 8/200
80/80 [==============================] - 1s 10ms/step - loss: 0.1992 - accuracy: 0.0799 - binary_accuracy: 0.9224
Epoch 9/200

80/80 [==============================] - 1s 6ms/step - loss: 0.1769 - accuracy: 0.0836 - binary_accuracy: 0.9319
Epoch 67/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1770 - accuracy: 0.0972 - binary_accuracy: 0.9313
Epoch 68/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1764 - accuracy: 0.0909 - binary_accuracy: 0.9323
Epoch 69/200
80/80 [==============================] - 1s 8ms/step - loss: 0.1770 - accuracy: 0.0977 - binary_accuracy: 0.9313
Epoch 70/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1753 - accuracy: 0.1015 - binary_accuracy: 0.9325
Epoch 71/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1739 - accuracy: 0.0900 - binary_accuracy: 0.9332
Epoch 72/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1746 - accuracy: 0.0859 - binary_accuracy: 0.9328
Epoch 73/200
80/80 [==============================] - 1s 9ms/step - loss: 0.1729 - accuracy: 0.0880 - binary_accuracy: 0.9342
Epoch

80/80 [==============================] - 1s 6ms/step - loss: 0.1560 - accuracy: 0.0983 - binary_accuracy: 0.9401
Epoch 131/200
80/80 [==============================] - 1s 6ms/step - loss: 0.1554 - accuracy: 0.0957 - binary_accuracy: 0.9410
Epoch 132/200
80/80 [==============================] - 1s 6ms/step - loss: 0.1582 - accuracy: 0.0892 - binary_accuracy: 0.9398
Epoch 133/200
80/80 [==============================] - 1s 6ms/step - loss: 0.1566 - accuracy: 0.0978 - binary_accuracy: 0.9410
Epoch 134/200
80/80 [==============================] - 1s 6ms/step - loss: 0.1564 - accuracy: 0.0968 - binary_accuracy: 0.9405
Epoch 135/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1593 - accuracy: 0.1059 - binary_accuracy: 0.9404
Epoch 136/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1556 - accuracy: 0.1032 - binary_accuracy: 0.9421
Epoch 137/200
80/80 [==============================] - 1s 6ms/step - loss: 0.1565 - accuracy: 0.1011 - binary_accuracy: 0.941

80/80 [==============================] - 1s 7ms/step - loss: 0.1403 - accuracy: 0.1050 - binary_accuracy: 0.9476
Epoch 195/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1394 - accuracy: 0.1047 - binary_accuracy: 0.9478
Epoch 196/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1402 - accuracy: 0.1043 - binary_accuracy: 0.9476
Epoch 197/200
80/80 [==============================] - 1s 6ms/step - loss: 0.1393 - accuracy: 0.1015 - binary_accuracy: 0.9475
Epoch 198/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1402 - accuracy: 0.1011 - binary_accuracy: 0.9476
Epoch 199/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1409 - accuracy: 0.1024 - binary_accuracy: 0.9467
Epoch 200/200
80/80 [==============================] - 1s 7ms/step - loss: 0.1386 - accuracy: 0.0985 - binary_accuracy: 0.9486


In [63]:
fully_connected_model.evaluate(X_test, Y_test)

16/32 [==============>...............] - ETA: 0s - loss: 1.4804 - accuracy: 0.0371 - binary_accuracy: 0.7126

2021-12-13 20:21:58.375146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 8ms/step - loss: 1.4836 - accuracy: 0.0370 - binary_accuracy: 0.7117


In [44]:
# This is a simple sequential model to demonstrate how this data can be used with a neural network
model = keras.Sequential()
model.add(Dense(128, input_dim=64, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='sigmoid'))

In [45]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", "binary_accuracy"]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_10 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


In [46]:
def evaluate_model(model, X, Y):
    loss, accuracy, binary_accuracy = model.evaluate(X_test, Y_test)
    print(f"Loss: {loss}\nBinary accuracy: {binary_accuracy}")
    return loss, accuracy, binary_accuracy

In [47]:
def fit_model(model, X, Y, n_epochs=50, batch_size=50):
    tensorboard_callback = new_tensorboard_run()
    model.fit(
        X, 
        Y, 
        epochs=200, 
        batch_size=50,
        callbacks=[tensorboard_callback]
    )

In [48]:
%tensorboard --logdir tensorboard_logs

Reusing TensorBoard on port 6006 (pid 60458), started 0:09:23 ago. (Use '!kill 60458' to kill it.)

In [49]:
fit_model(model, X_train, Y_train)

2021-12-13 20:03:11.680878: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:03:11.680897: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:03:11.684323: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/200
 3/80 [>.............................] - ETA: 2s - loss: 0.7074 - accuracy: 0.0122 - binary_accuracy: 0.4803 

2021-12-13 20:03:12.196320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 20:03:12.308744: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 20:03:12.308755: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 20:03:12.352273: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 20:03:12.352725: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 20:03:12.353454: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/20211213-200311/train/plugins/profile/2021_12_13_20_03_12
2021-12-13 20:03:12.353923: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/20211213-200311/train/plugins/profile/2021_12_13_20_03_12/E

80/80 [==============================] - 1s 7ms/step - loss: 0.6295 - accuracy: 0.0085 - binary_accuracy: 0.6512
Epoch 2/200
80/80 [==============================] - 0s 6ms/step - loss: 0.5594 - accuracy: 0.0069 - binary_accuracy: 0.7189
Epoch 3/200
80/80 [==============================] - 0s 6ms/step - loss: 0.5312 - accuracy: 0.0119 - binary_accuracy: 0.7312
Epoch 4/200
80/80 [==============================] - 0s 6ms/step - loss: 0.5102 - accuracy: 0.0151 - binary_accuracy: 0.7415
Epoch 5/200
80/80 [==============================] - 0s 6ms/step - loss: 0.5012 - accuracy: 0.0112 - binary_accuracy: 0.7450
Epoch 6/200
80/80 [==============================] - 1s 6ms/step - loss: 0.4900 - accuracy: 0.0154 - binary_accuracy: 0.7516
Epoch 7/200
80/80 [==============================] - 0s 6ms/step - loss: 0.4828 - accuracy: 0.0130 - binary_accuracy: 0.7563
Epoch 8/200
80/80 [==============================] - 0s 6ms/step - loss: 0.4805 - accuracy: 0.0150 - binary_accuracy: 0.7574
Epoch 9/200


80/80 [==============================] - 0s 6ms/step - loss: 0.3205 - accuracy: 0.0470 - binary_accuracy: 0.8658
Epoch 67/200
80/80 [==============================] - 1s 6ms/step - loss: 0.3209 - accuracy: 0.0476 - binary_accuracy: 0.8643
Epoch 68/200
80/80 [==============================] - 0s 6ms/step - loss: 0.3177 - accuracy: 0.0521 - binary_accuracy: 0.8664
Epoch 69/200
80/80 [==============================] - 0s 6ms/step - loss: 0.3183 - accuracy: 0.0471 - binary_accuracy: 0.8659
Epoch 70/200
80/80 [==============================] - 0s 6ms/step - loss: 0.3187 - accuracy: 0.0479 - binary_accuracy: 0.8660
Epoch 71/200
80/80 [==============================] - 0s 6ms/step - loss: 0.3158 - accuracy: 0.0588 - binary_accuracy: 0.8680
Epoch 72/200
80/80 [==============================] - 0s 6ms/step - loss: 0.3112 - accuracy: 0.0424 - binary_accuracy: 0.8697
Epoch 73/200
80/80 [==============================] - 0s 6ms/step - loss: 0.3130 - accuracy: 0.0473 - binary_accuracy: 0.8693
Epoch

80/80 [==============================] - 1s 6ms/step - loss: 0.2449 - accuracy: 0.0665 - binary_accuracy: 0.9021
Epoch 131/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2483 - accuracy: 0.0687 - binary_accuracy: 0.9005
Epoch 132/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2467 - accuracy: 0.0823 - binary_accuracy: 0.9017
Epoch 133/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2468 - accuracy: 0.0700 - binary_accuracy: 0.9017
Epoch 134/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2456 - accuracy: 0.0682 - binary_accuracy: 0.9030
Epoch 135/200
80/80 [==============================] - 1s 6ms/step - loss: 0.2446 - accuracy: 0.0669 - binary_accuracy: 0.9024
Epoch 136/200
80/80 [==============================] - 0s 6ms/step - loss: 0.2459 - accuracy: 0.0737 - binary_accuracy: 0.9015
Epoch 137/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2410 - accuracy: 0.0699 - binary_accuracy: 0.90

80/80 [==============================] - 1s 7ms/step - loss: 0.2088 - accuracy: 0.0707 - binary_accuracy: 0.9179
Epoch 195/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2059 - accuracy: 0.0845 - binary_accuracy: 0.9191
Epoch 196/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2065 - accuracy: 0.0733 - binary_accuracy: 0.9199
Epoch 197/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2049 - accuracy: 0.0736 - binary_accuracy: 0.9205
Epoch 198/200
80/80 [==============================] - 1s 6ms/step - loss: 0.2054 - accuracy: 0.0915 - binary_accuracy: 0.9196
Epoch 199/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2042 - accuracy: 0.0918 - binary_accuracy: 0.9199
Epoch 200/200
80/80 [==============================] - 1s 7ms/step - loss: 0.2060 - accuracy: 0.0878 - binary_accuracy: 0.9194


In [50]:
evaluate_model(model, X_test, Y_test)

17/32 [==============>...............] - ETA: 0s - loss: 0.9442 - accuracy: 0.0368 - binary_accuracy: 0.7159

2021-12-13 20:05:02.504193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 8ms/step - loss: 0.9343 - accuracy: 0.0330 - binary_accuracy: 0.7155
Loss: 0.934349000453949
Binary accuracy: 0.7155312895774841


(0.934349000453949, 0.032999999821186066, 0.7155312895774841)

## Visualising the output
Now both the raw output of the model and the binarised version can be displayed interactively.

In [18]:
# Functions to manipulate the output
def predict_val(model, val):
    prediction = model.predict(val)
    return np.where(prediction > 0.5, 1, 0)

def predict_raw(model, val):
    return model.predict(val)  

def find_diff(y, pred):
    Y_instance = y
    return np.where(pred == Y_instance, 1, 0)

In [19]:
# Functions to draw the labels and predictions.

sns.set_theme(style="white")
sns.color_palette("crest", as_cmap=True)
sns.set_palette("crest")

def plot_sns(val):
    # From https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    sns.heatmap(val.reshape(8,8))
    plt.show()

def plot(val):
    cmap = colors.ListedColormap(['white', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val.reshape(8, 8), cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    plt.show()

def plot_diff(val):
    cmap = colors.ListedColormap(['red', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val.reshape(8,8) , cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    plt.show()

In [20]:
def generate_plots(
            n, 
            model = None,
            x = None,
            y = None,
            target=True, 
            prediction=True, 
            raw_prediction=True, 
            diff=True
        ):
    if target:         
        plot(y[n].reshape(1, 64))    
    if prediction:     
        plot(predict_val(model, x[n:n+1]))
    if raw_prediction: 
        plot_sns(predict_raw(model, x[n:n+1]))
    if diff:          
        pred = predict_val(model, x[n:n+1])
        plot_diff(find_diff(y[n:n+1], pred))

In [21]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots,
    model = fixed(model),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

This widget lets us look through the validation set predictions and compare what the model is outputting compared to what the equivilant label is. What we can begin to see from here is that the model does have some predictive power on pieces which do not move great distances up and down the board. However for pieces that can travel the entire diagonal, vertical or horizontal distance of the board (such as rooks, bishops and queens) there is a high margin of error and these motions are rarely predicted at all, indicating they are tough to learn.

This may also be indicitve of some bias in the training data - almost every position has pawns which can be moved in it whereas the major pieces are more often closed in and obscured behind other pieces. This imbalance may contribute to not learning how the pieces move properly.

# Convolutional design

We can also approach this problem with a convolutional network. These are designed for image processing and operate by sliding a kernel over an image, performing a "convolution" of the kernel on the data. It is particuarly good at identifying patterns, so let us test this out!

## Data manipulation

For this model, the data will be separated into two layers; one for all the white pieces and one for the black pieces. The target will remain unchanged and there will still be no distinction between the pieces.

To get the data in the correct format, we can construct a $8\times8\times2$ numpy array to feed our model. The pieces which are moving will be provided in the 0th index of axis 2 (the array is structured [axis0, axis1, axis2]) and those not moving will be at index 1.

In [22]:
conv_move_list = [list({i for i in board.moves.all_valid}) for board in board_list]

conv_position_list = [
    (
        dict(board.loc_map), 
        board.to_move
    ) 
    for board in board_list
]

In [23]:
# Encode the location of any piece as a 1, and all other positions as 0
conv_feature = []
for instance in conv_position_list:
    # Set up empty array
    array = np.zeros((8, 8, 2))
    
    # Assign white and black to their correct layers
    if instance[1] == WHITE:
        # White to move
        WHITE_LAYER = 1
        BLACK_LAYER = 0
    else:
        # Black to move
        BLACK_LAYER = 1
        WHITE_LAYER = 0
    
    # Mark the correct locations in the matrix
    for location, piece in instance[0].items():
        if piece.colour == WHITE:
            array[location.i, location.j, WHITE_LAYER] = 1
        else:
            array[location.i, location.j, BLACK_LAYER] = 1

    conv_feature.append(array)

In [24]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
conv_target = []
for instance in conv_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    conv_target.append(array.flatten())

In [25]:
X = np.array(conv_feature)
Y = np.array(conv_target)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### Training a model

Now the data is in a format that contains much more information in a way that convolutional netoworks can process, we can define a convolutional model architecture (again using the Keras sequential API) to verify if this model can perform any better.

In [26]:
model = keras.Sequential()
model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(8, 8, 2)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128))
model.add(Dense(64, activation='sigmoid'))

In [27]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", "binary_accuracy"]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 6, 8)           152       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 8)           584       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8448      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 64)               

In [28]:
fit_model(model, X_train, Y_train)

Epoch 1/200


2021-12-13 19:53:49.243623: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 19:53:49.243634: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 19:53:49.244130: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


 1/80 [..............................] - ETA: 1:50 - loss: 0.6935 - accuracy: 0.0400 - binary_accuracy: 0.4937

2021-12-13 19:53:50.536808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 19:53:50.681989: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 19:53:50.682000: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 19:53:50.713677: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 19:53:50.714259: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 19:53:50.715027: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/20211213-195349/train/plugins/profile/2021_12_13_19_53_50
2021-12-13 19:53:50.715528: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/20211213-195349/train/plugins/profile/2021_12_13_19_53_50/E

80/80 [==============================] - 2s 9ms/step - loss: 0.6305 - accuracy: 0.0071 - binary_accuracy: 0.6737
Epoch 2/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5423 - accuracy: 0.0056 - binary_accuracy: 0.7241
Epoch 3/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5263 - accuracy: 0.0042 - binary_accuracy: 0.7364
Epoch 4/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5180 - accuracy: 0.0027 - binary_accuracy: 0.7423
Epoch 5/200
80/80 [==============================] - 1s 9ms/step - loss: 0.5139 - accuracy: 0.0049 - binary_accuracy: 0.7480
Epoch 6/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5046 - accuracy: 0.0091 - binary_accuracy: 0.7538
Epoch 7/200
80/80 [==============================] - 1s 8ms/step - loss: 0.4881 - accuracy: 0.0119 - binary_accuracy: 0.7663
Epoch 8/200
80/80 [==============================] - 1s 8ms/step - loss: 0.4828 - accuracy: 0.0152 - binary_accuracy: 0.7696
Epoch 9/200


80/80 [==============================] - 1s 8ms/step - loss: 0.3512 - accuracy: 0.0366 - binary_accuracy: 0.8417
Epoch 67/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3482 - accuracy: 0.0303 - binary_accuracy: 0.8438
Epoch 68/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3468 - accuracy: 0.0329 - binary_accuracy: 0.8439
Epoch 69/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3439 - accuracy: 0.0364 - binary_accuracy: 0.8459
Epoch 70/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3462 - accuracy: 0.0320 - binary_accuracy: 0.8453
Epoch 71/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3443 - accuracy: 0.0295 - binary_accuracy: 0.8450
Epoch 72/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3439 - accuracy: 0.0360 - binary_accuracy: 0.8457
Epoch 73/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3417 - accuracy: 0.0295 - binary_accuracy: 0.8454
Epoch

80/80 [==============================] - 1s 8ms/step - loss: 0.3160 - accuracy: 0.0399 - binary_accuracy: 0.8608
Epoch 131/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3158 - accuracy: 0.0408 - binary_accuracy: 0.8613
Epoch 132/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3170 - accuracy: 0.0393 - binary_accuracy: 0.8605
Epoch 133/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3163 - accuracy: 0.0425 - binary_accuracy: 0.8607
Epoch 134/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3112 - accuracy: 0.0406 - binary_accuracy: 0.8635
Epoch 135/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3127 - accuracy: 0.0440 - binary_accuracy: 0.8624
Epoch 136/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3122 - accuracy: 0.0344 - binary_accuracy: 0.8630
Epoch 137/200
80/80 [==============================] - 1s 8ms/step - loss: 0.3111 - accuracy: 0.0355 - binary_accuracy: 0.862

80/80 [==============================] - 1s 10ms/step - loss: 0.3005 - accuracy: 0.0458 - binary_accuracy: 0.8681
Epoch 195/200
80/80 [==============================] - 1s 12ms/step - loss: 0.2974 - accuracy: 0.0355 - binary_accuracy: 0.8700
Epoch 196/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2972 - accuracy: 0.0426 - binary_accuracy: 0.8708
Epoch 197/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2985 - accuracy: 0.0393 - binary_accuracy: 0.8692
Epoch 198/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2968 - accuracy: 0.0389 - binary_accuracy: 0.8704
Epoch 199/200
80/80 [==============================] - 1s 9ms/step - loss: 0.3001 - accuracy: 0.0375 - binary_accuracy: 0.8689
Epoch 200/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2995 - accuracy: 0.0418 - binary_accuracy: 0.8686


In [29]:
evaluate_model(model, X_test, Y_test)

19/32 [================>.............] - ETA: 0s - loss: 0.4799 - accuracy: 0.0247 - binary_accuracy: 0.7965

2021-12-13 19:56:18.321723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 1s 7ms/step - loss: 0.4894 - accuracy: 0.0250 - binary_accuracy: 0.7938
Loss: 0.48936188220977783
Binary accuracy: 0.7938281893730164


(0.48936188220977783, 0.02500000037252903, 0.7938281893730164)

The binary accuracy of this model does outperform the previous dense network used before, which is a promising result for this technique.

In [30]:
interact(
    generate_plots,
    model = fixed(model),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

From these modified visualisation functions it is clear that there is greater performance by this model. The added implicit information on which pieces are moving, combined with the use of convolutional filters to properly extract features from the training data. Seeing a jump in binary accuracy as well as in many cases much more confident predictions from the model is indicitive that this approach is superior, due to the convolutional feature extraction layers

The behaviour of long range pieces is however still an issue for these systems.

# Convolutional network with more layers

It is now not unreasonable to think that the networks performance would be increased significantly if it were to be aware of which pieces are at which location; this is after all the most important deciding factor as to how they may move.

An approach to integrating this data is the following architecture;
- Input data (8 x 8 x 6), where there is one index on axis 2 for each of the five piece types (King, Queen, Rook, Knight, Bishop, Pawn) for the moving pieces, and another for the following
- The final layer of the input tensor will contain the position of the pieces which are not to move. Since the information on which type of piece these are is generally less important to predicting the moves, for now this can be left out.
- Additionally, passing the fully connected layer some information about whether the position is check for example may increase accuracy some more, since it will be able to tell when certain moves which usually are valid would not be valid. This information needs to reach the fully connected network but is not part of the convolutional network, so for this section the Keras functional API will be more appropriate as it offers more control.





## Data processing

In [31]:
# Encode the location of any piece as a 1, and all other positions as 0
conv_feature = []
piece_index_map = {
    'K': 0,
    'Q': 1,
    'R': 2,
    'B': 3,
    'N': 4,
    'P': 5
}
for instance in conv_position_list:
    # Set up empty array
    array = np.zeros((8, 8, 7))
    
    # Assign white and black to their correct layers
    if instance[1] == WHITE:
        # White to move
        MOVING_COLOUR = 1
    else:
        # Black to move
        MOVING_COLOUR = 0
    
    # Mark the correct locations in the matrix
    for location, piece in instance[0].items():
        if piece.colour == MOVING_COLOUR:
            array[location.i, location.j, piece_index_map[piece.kind]] = 1
        else:
            array[location.i, location.j, 6] = 1

    conv_feature.append(array)

In [32]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
conv_target = []
for instance in conv_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    conv_target.append(array.flatten())

In [33]:
X = np.array(conv_feature)
Y = np.array(conv_target)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Create a model

In [34]:
inputs = keras.Input(shape=(8, 8, 7))
cnv = Conv2D(10, (3, 3), activation='relu')(inputs)
cnv = Conv2D(10, (3, 3), activation='relu')(cnv)
cnv = Conv2D(10, (3, 3), activation='relu')(cnv)
cnv = Flatten()(cnv)
cnv = Dense(256, activation='relu')(cnv)
cnv = Dense(128, activation='relu')(cnv)
outputs = Dense(64, activation='sigmoid')(cnv)

model = keras.Model(inputs=inputs, outputs=outputs, name="ConvNet7l")


In [35]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", "binary_accuracy"]
)
model.summary()

Model: "ConvNet7l"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 7)]         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 10)          640       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 10)          910       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 10)          910       
_________________________________________________________________
flatten_1 (Flatten)          (None, 40)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               10496     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32

In [36]:
fit_model(model, X_train, Y_train)

Epoch 1/200


2021-12-13 19:56:19.189423: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 19:56:19.189433: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 19:56:19.189477: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


 1/80 [..............................] - ETA: 1:01 - loss: 0.6936 - accuracy: 0.0000e+00 - binary_accuracy: 0.4903

2021-12-13 19:56:19.860191: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-13 19:56:20.010670: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-13 19:56:20.010682: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-13 19:56:20.056824: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 19:56:20.057260: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-13 19:56:20.057984: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/20211213-195619/train/plugins/profile/2021_12_13_19_56_20
2021-12-13 19:56:20.058433: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/20211213-195619/train/plugins/profile/2021_12_13_19_56_20/E

80/80 [==============================] - 2s 10ms/step - loss: 0.6386 - accuracy: 0.0080 - binary_accuracy: 0.6607
Epoch 2/200
80/80 [==============================] - 1s 9ms/step - loss: 0.5700 - accuracy: 0.0083 - binary_accuracy: 0.7096
Epoch 3/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5310 - accuracy: 0.0026 - binary_accuracy: 0.7320
Epoch 4/200
80/80 [==============================] - 1s 11ms/step - loss: 0.5217 - accuracy: 0.0059 - binary_accuracy: 0.7401
Epoch 5/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5142 - accuracy: 0.0038 - binary_accuracy: 0.7442
Epoch 6/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5013 - accuracy: 0.0054 - binary_accuracy: 0.7536
Epoch 7/200
80/80 [==============================] - 1s 10ms/step - loss: 0.5016 - accuracy: 0.0132 - binary_accuracy: 0.7524
Epoch 8/200
80/80 [==============================] - 1s 10ms/step - loss: 0.4911 - accuracy: 0.0220 - binary_accuracy: 0.7590
Epoch 

80/80 [==============================] - 1s 10ms/step - loss: 0.3040 - accuracy: 0.0299 - binary_accuracy: 0.8674
Epoch 67/200
80/80 [==============================] - 1s 10ms/step - loss: 0.3025 - accuracy: 0.0364 - binary_accuracy: 0.8684
Epoch 68/200
80/80 [==============================] - 1s 10ms/step - loss: 0.3012 - accuracy: 0.0344 - binary_accuracy: 0.8687
Epoch 69/200
80/80 [==============================] - 1s 11ms/step - loss: 0.2967 - accuracy: 0.0383 - binary_accuracy: 0.8717
Epoch 70/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2987 - accuracy: 0.0340 - binary_accuracy: 0.8704
Epoch 71/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2953 - accuracy: 0.0363 - binary_accuracy: 0.8719
Epoch 72/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2964 - accuracy: 0.0373 - binary_accuracy: 0.8716
Epoch 73/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2910 - accuracy: 0.0390 - binary_accuracy: 0.87

80/80 [==============================] - 1s 9ms/step - loss: 0.2389 - accuracy: 0.0385 - binary_accuracy: 0.8991
Epoch 131/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2362 - accuracy: 0.0402 - binary_accuracy: 0.9012
Epoch 132/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2360 - accuracy: 0.0500 - binary_accuracy: 0.9007
Epoch 133/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2352 - accuracy: 0.0413 - binary_accuracy: 0.9015
Epoch 134/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2318 - accuracy: 0.0400 - binary_accuracy: 0.9030
Epoch 135/200
80/80 [==============================] - 1s 10ms/step - loss: 0.2326 - accuracy: 0.0446 - binary_accuracy: 0.9023
Epoch 136/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2321 - accuracy: 0.0486 - binary_accuracy: 0.9029
Epoch 137/200
80/80 [==============================] - 1s 9ms/step - loss: 0.2281 - accuracy: 0.0415 - binary_accuracy: 0.

80/80 [==============================] - 1s 9ms/step - loss: 0.2017 - accuracy: 0.0557 - binary_accuracy: 0.9170
Epoch 195/200
80/80 [==============================] - 1s 8ms/step - loss: 0.1985 - accuracy: 0.0440 - binary_accuracy: 0.9178
Epoch 196/200
80/80 [==============================] - 1s 8ms/step - loss: 0.1992 - accuracy: 0.0562 - binary_accuracy: 0.9181
Epoch 197/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2025 - accuracy: 0.0487 - binary_accuracy: 0.9157
Epoch 198/200
80/80 [==============================] - 1s 8ms/step - loss: 0.1974 - accuracy: 0.0498 - binary_accuracy: 0.9186
Epoch 199/200
80/80 [==============================] - 1s 8ms/step - loss: 0.2024 - accuracy: 0.0473 - binary_accuracy: 0.9163
Epoch 200/200
80/80 [==============================] - 1s 8ms/step - loss: 0.1991 - accuracy: 0.0504 - binary_accuracy: 0.9181


In [38]:
evaluate_model(model, X_test, Y_test)

18/32 [===============>..............] - ETA: 0s - loss: 0.5048 - accuracy: 0.0243 - binary_accuracy: 0.8251

2021-12-13 19:59:27.569484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 7ms/step - loss: 0.5133 - accuracy: 0.0260 - binary_accuracy: 0.8233
Loss: 0.5132701992988586
Binary accuracy: 0.8233281373977661


(0.5132701992988586, 0.026000000536441803, 0.8233281373977661)

## Visualise the output

In [37]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots,
    model = fixed(model),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

Although this model performs approximately on par with the original fully connected network, we can see dramatic improvments in the ability to handle the motion of "long range" pieces - the model now has the ability to learn about which pieces can move in which axis. 